In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
person = validation_data_input.get_data('person',config['survey_dir'])
hh = validation_data_input.get_data('household',config['survey_dir'])

In [3]:
# parcel land use data
df_parcel = pd.read_csv(os.path.join(model_dir,r'outputs\landuse\buffered_parcels.txt'),
                        delim_whitespace=True,
                        usecols=['parcelid','emptot_1','hh_1'])

In [4]:
# Load parcel geography lookups
parcel_geog = pd.read_sql_table(
    'parcel_'+config['base_year']+'_geography',
    'sqlite:///'+model_dir+'/inputs/db/soundcast_inputs.db',
    columns=['ParcelID','CountyName','rg_proposed']
    )

In [5]:
hh = hh.merge(df_parcel, left_on='hhparcel', right_on='parcelid', how='left')

In [6]:
# Group income, hh density, and employment density into 4 groups
var_group = hh.loc[hh['source'] == 'model',['hhincome','emptot_1','hh_1']].quantile([.125, .25, .50, .75])

# var_group

In [7]:
hh['hhincome_group'] = pd.cut(hh['hhincome'],bins=[-9999999.0] + var_group['hhincome'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])
# add hh density groups
hh['hh_density_group'] = pd.cut(hh['hh_1'],bins=[-9999999.0] + var_group['hh_1'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])
# add employment density groups
hh['emp_density_group'] = pd.cut(hh['emptot_1'],bins=[-9999999.0] + var_group['emptot_1'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])


In [8]:
person = person.merge(hh, on=['hhno'], how='left')

In [9]:
person = person.merge(parcel_geog, left_on='pwpcl', right_on='ParcelID', how='left')
person = person.merge(parcel_geog, left_on='hhparcel', right_on='ParcelID', how='left', suffixes=['_work', '_home'])

In [10]:
person.drop('source_x', axis=1, inplace=True)
person.rename(columns={'source_y': 'source'}, inplace=True)

In [11]:
df_workers = person.loc[person['pwtyp']>0, :].copy()

In [12]:
df = df_workers.loc[df_workers['pwaudist'] > 0, :].copy()
df['wt_dist'] = df['pwaudist']*df['psexpfac']
df = df.groupby(['source'])[['wt_dist','psexpfac']].sum().reset_index()
df['Mean Distance to Work by Car (pwaudist)'] = df['wt_dist']/df['psexpfac']
df[['source','Mean Distance to Work by Car (pwaudist)']]

,source,Mean Distance to Work by Car (pwaudist)
0,model,11.154587
1,survey,11.143076


In [13]:

def plot_work_location(df:pd.DataFrame, var:str, title_cat:str, sub_name:str):
    df = df[df['pwaudist'] > 0].copy()
    df['wt_dist'] = df['pwaudist']*df['psexpfac']
    df_plot = df.groupby(['source',var])[['wt_dist','psexpfac']].sum().reset_index()
    df_plot['average_wt_pwaudist'] = df_plot['wt_dist']/df_plot['psexpfac']
    df_plot

    df_plot_ct = df.groupby(['source',var])[['wt_dist','psexpfac']].count().reset_index(). \
    rename(columns={'psexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',var])

    fig = px.bar(df_plot, x=var, y=['average_wt_pwaudist', 'source'], color="source",
                    barmode="group",
                     hover_data=['sample count'],
                    title="Work Distance by "+ title_cat)
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text=sub_name)
    fig.update_layout(height=400, width=800, font=dict(size=11),
                      yaxis=dict(tickformat=".2f"))
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".2f"))
    fig.show()

## Work Location by Workplace Geography

In [14]:
plot_work_location(df_workers, 'CountyName_work', 'Work County', 'County')

In [15]:
plot_work_location(df_workers, 'rg_proposed_work', 'Regional Geography', 'Regional Geography')

## Workplace by Home Geography

In [16]:
plot_work_location(df_workers, 'CountyName_home', 'Home County', 'County')

In [17]:
plot_work_location(df_workers, 'rg_proposed_home', 'Home Regional Geography', 'Regional Geography')

## Workplace Location by Person/Household Characteristics

In [18]:
plot_work_location(df_workers, 'pwtyp', 'Employment Type', 'employment type')

In [19]:
plot_work_location(df_workers, 'hhincome_group', 'Income', 'Income Group')

In [20]:
plot_work_location(df_workers, 'hh_density_group', 'Household Density', 'Household Density Group')

In [21]:
plot_work_location(df_workers, 'emp_density_group', 'Employment Density', 'Employment Density Group')